In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import time
from dotenv import load_dotenv

In [2]:
from openai import OpenAI
load_dotenv()
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [3]:
user_train_path = '../train_valid_test/user_train.pkl'
user_valid_path = '../train_valid_test/user_valid.pkl'
user_test_path = '../train_valid_test/user_test.pkl'

In [4]:
with open(user_train_path, 'rb') as f:
    user_train_dict = pickle.load(f)

In [5]:
with open(user_valid_path, 'rb') as f:
    user_valid_dict = pickle.load(f)

In [6]:
with open(user_test_path, 'rb') as f:
    user_test_dict = pickle.load(f)

In [7]:
with open('./sem_sim_dataset/sem_sim_valid.pkl', 'rb') as f:
    user_sem_sim_valid_data = pickle.load(f)
print(len(user_sem_sim_valid_data), user_sem_sim_valid_data[1])

6040 [1022, 1270, 919, 938, 2018, 1028, 661, 3114, 1029, 2321, 1197, 594, 1, 588, 2687, 595, 2294, 783, 2355, 1907]


In [8]:
with open('./sem_sim_dataset/sem_sim_test.pkl', 'rb') as f:
    user_sem_sim_test_data = pickle.load(f)
print(len(user_sem_sim_test_data), user_sem_sim_test_data[1])

6040 [1721, 1022, 919, 2018, 1028, 1246, 661, 531, 3114, 1029, 2321, 1197, 594, 1, 588, 2687, 595, 783, 1907, 1566]


In [9]:
user_id = 1
print(user_train_dict[user_id])
print(user_valid_dict[user_id])
print(user_test_dict[user_id])

[[1961, 2028, 3105, 938, 1962, 1035, 150, 2018, 1028, 1097, 914, 2797, 1287, 2762, 1246, 661, 2918, 531, 3114, 2791, 1029, 2321, 1197, 594, 2398, 1545, 527, 1, 588, 2687, 745, 595, 2294, 783, 2355, 1907]]
[1566]
[48]


In [10]:
len(user_train_dict[user_id][0])

36

In [11]:
user_profile_path = '../user_profile/cleaned_user_profile_mixtral.pkl'
with open(user_profile_path, 'rb') as f:
    user_profile_dict = pickle.load(f)
print(user_profile_dict[user_id])


            The user has a diverse taste in movies, with a preference for thought-provoking and emotionally engaging films. They enjoy dramas that explore complex themes such as mental illness, racial injustice, and the meaning of life. The user is drawn to movies that challenge their perspective and provoke deep thought, often favoring those that tackle serious and mature subjects.
            
            In addition to dramas, the user also has a soft spot for classic Disney films, as evidenced by their fondness for Cinderella. They appreciate the enchanting storytelling and timeless charm of these animated tales.
            
            The user's taste in movies also extends to comedies, as seen in their appreciation for the hilarious antics of Wallace & Gromit and the dark humor of Fargo. They are also drawn to adventure films, as demonstrated by their love for Star Wars: Episode IV - A New Hope and Back to the Future.
            
            However, the user is not a fan of 

In [12]:
movie_summary_df = pd.read_csv('../movie_summary_25_words.csv')
movie_summary_df.head()

,movie_id,name,cleaned_genre,year,summary
0,1,Toy Story,"Animation,Children's,Comedy",1995,Toy Story is a groundbreaking animated film th...
1,2,Jumanji,"Adventure,Children's,Fantasy",1995,"""Jumanji (1995) is a thrilling adventure film ..."
2,3,Grumpier Old Men,"Comedy,Romance",1995,"""Grumpier Old Men is a hilarious and heartwarm..."
3,4,Waiting to Exhale,"Comedy,Drama",1995,"""Waiting to Exhale"" (1995) is a heartfelt dram..."
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II (1995) is a heartw...


In [13]:
print(movie_summary_df[movie_summary_df['movie_id'] == 265])

     movie_id                                               name  \
262       265  Like Water for Chocolate (Como agua para choco...   

     cleaned_genre  year                                            summary  
262  Drama,Romance  1992  "Like Water for Chocolate is a visually stunni...  


In [14]:
user_df = pd.read_csv('../users.csv')
user_df.head()

,user_id,user_profile
0,1,female k-12 student under 18 years of age
1,2,male self-employed above 56 years of age
2,3,male scientist between 25 to 34 years of age
3,4,male executive/managerial between 45 to 49 yea...
4,5,male writer between 25 to 34 years of age


In [15]:
ratings_df = pd.read_csv('../ratings.csv')
ratings_df.head()

,user_id,movie_id,rating
0,1,3186,4
1,1,1721,4
2,1,1022,5
3,1,1270,5
4,1,2340,3


In [22]:
crt_sem_valid_zeroshot_dataset = dict()
cnt = 0
for user, user_profile in user_profile_dict.items():
    # if cnt < 2:
    #     cnt += 1
    #     continue
    cnt += 1
    # print(f"User: {user}")
    prompt = """
    You are an expert movie critic. You are provided with the user profile and list of movies the user has watched and whether the user likes it or not.
    User Profile -
    """
    ### Create the user profile
    final_user_profile = "The user is a "+user_df[user_df['user_id'] == user]['user_profile'].values[0]+user_profile+"\n"
    # print(final_user_profile)
    prompt += final_user_profile
    
    prompt += """
   User watched the following movies in the given order. List of movies and their description -
    """
    # print(prompt)
    ### Create the like/dislike list of last 20 movies
    # print(user_train_dict[user][0][-20:])
    user_rating_dict = ratings_df[ratings_df['user_id'] == user]
    # print(user_rating_dict)
    liked_movies = []
    disliked_movies = []
    # train_valid_movie_list = user_train_dict[user][0][-19:]
    # train_valid_movie_list.append(user_valid_dict[user][0])
    for movie in user_sem_sim_valid_data[user]:
        # print(movie)
        user_movie_rating = user_rating_dict[user_rating_dict['movie_id'] == movie]['rating'].values[0]
        # print(movie, user_movie_rating)
        movie_details = movie_summary_df[movie_summary_df['movie_id'] == movie]
        if user_movie_rating >= 3:
            if movie != user_valid_dict[user][0]:
                prompt += "\nLiked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " + movie_details['summary'].values[0] + ","
            else:
                ### For the last movie - ending with a . and starting new line
                prompt += "\nLiked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0] + "\n"
        else:
            if movie != user_valid_dict[user][0]:
                prompt += "\nDisliked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0] + ","
            else:
                ### For the last movie - ending with a . and starting new line
                prompt += "\nDisliked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0] + "\n"

    ### To predict
    # print(user_valid_dict[user][0])
    movie_details = movie_summary_df[movie_summary_df['movie_id'] == user_valid_dict[user][0]]
    prompt += f"""
    The user has watched the above movies chronologically in past. The given list of movies are the 20 most similar movies to the next movie -
    {movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0]}.
    Analyze all the information given in order. Do not use any information not mentioned above. Summarize the user's behavior from the given list of movies and identify the movie characteristics that he likes and dislikes.
    """
    # print("Test Movie Rating:", user_rating_dict[user_rating_dict['movie_id'] == user_valid_dict[user][0]]['rating'].values[0])
    # prompt += movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0]
    prompt += f"""
    As an expert movie critic and recommender, provide the reasoning why the user will like or dislike {movie_details['name'].values[0]} by analyzing user behavior, movie watching history and explain your reasoning.
    According to your reasoning, predict whether the user will like the movie or not. Finally answer with a Yes or No in the following format -
    Prediction: Yes or No
    """
    # print(f"Prompt length: {len(prompt.split())}", prompt)
    crt_sem_valid_zeroshot_dataset[user] = prompt
    # break

In [23]:
print(len(crt_sem_valid_zeroshot_dataset))

6040


In [24]:
crt_sem_valid_zeroshot_dataset[3246]

'\n    You are an expert movie critic. You are provided with the user profile and list of movies the user has watched and whether the user likes it or not.\n    User Profile -\n    The user is a female academic/educator between 35 to 44 years of age\n            The user has a diverse taste in movies, with a preference for complex and thought-provoking films. They enjoy movies that push the boundaries of sanity and freedom, as seen in "One Flew Over the Cuckoo\'s Nest," and morally ambiguous characters, as depicted in "Miller\'s Crossing." The user is drawn to political dramas, as evidenced by their interest in "The Candidate," and adventure films like "Raiders of the Lost Ark."\n            The user also has a soft spot for movies that explore the human condition, such as "Butterfly (La Lengua de las Mariposas)," which delves into the bond between a young boy and his teacher during the Spanish Civil War. They are also interested in movies that intertwine past and present, as seen in "

In [25]:
crt_sem_valid_zeroshot_dataset_path = './sem_sim_dataset/sem_ctr_valid_zeroshot_dataset_2.pkl'
with open(crt_sem_valid_zeroshot_dataset_path, 'wb') as f:
    pickle.dump(crt_sem_valid_zeroshot_dataset, f)
    f.close()

In [37]:
# with open(crt_valid_zeroshot_dataset_path, 'rb') as f:
#     crt_valid_zeroshot_dataset_temp = pickle.load(f)
# print(len(crt_valid_zeroshot_dataset_temp), crt_valid_zeroshot_dataset_temp[1])

In [29]:
content = prompt
print(crt_sem_valid_zeroshot_dataset[9])
print('*'*100)
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "user",
      "content": crt_sem_valid_zeroshot_dataset[9]
    }
  ],
  temperature=0.1,
  max_tokens=1024,
  top_p=0.9
)


response = response.choices[0].message.content
print(response)
len(response.split(' '))


    You are an expert movie critic. You are provided with the user profile and list of movies the user has watched and whether the user likes it or not.
    User Profile -
    The user is a male technician/engineer between 25 to 34 years of age
            The user's preferred movie genres include drama, animation, war, biographical, sports, comedy, and thriller films. They have a strong affinity for movies based on true stories, such as "Hurricane" and "Schindler's List," as well as historical dramas like "The Godfather: Part II" and "GoodFellas." The user also enjoys a mix of light-hearted and heavy films, with a taste for both the comedic, like "Meet the Parents," and the thrilling, such as "Silence of the Lambs" and "The Usual Suspects."
            
            In terms of animation, the user leans towards stop-motion films, with "Chicken Run" and "Wrong Trousers" among their favorites. They also appreciate a blend of humor and adventure in their animated movies.
            
   

151